In [ ]:
import json
import sys
import os
sys.path.insert(0, '../ghtesting')

import pandas as pd
from util import *

In [ ]:
repos = get_repos()

In [ ]:
for repo in repos:
    latest = get_latest_report(repo)
    earliest = get_earliest_report(repo)
    
    if latest is None or earliest is None:
        continue

    print(repo.name)
    
    print("Web framework:", get_webframework(repo))
    print("Main branch:", get_branch(repo))
    
    
    print("Latest report:", latest['timestamp'])
    print("Contributors:", len(get_contributors(repo, latest)))
    print("Earliest report:", earliest['timestamp'])
    print("Contributors:", len(get_contributors(repo, earliest)))

    print()

In [ ]:
get_metrics(get_latest_report(repos[1]))

In [ ]:
get_metrics(get_earliest_report(repos[1]))